In [1]:
import sys
import datetime 
from dateutil.relativedelta import relativedelta
import random
from faker import Faker
import csv
import pandas as pd
import numpy as np
import json
import itertools

In [2]:
class AutoVivification(dict):
    """Implementation of perl's autovivification feature."""
    def __getitem__(self, item):
        try:
            return dict.__getitem__(self, item)
        except KeyError:
            value = self[item] = type(self)()
            return value

In [435]:
def return_rangepicks(childct,itemdivs,limit):
    childct
    start=itemdivs[0]
    stop=itemdivs[1]

    rangepicks=[]
    
    if childct<=2:
        rangepicks=itemdivs
    else:
        step=int((stop-(start-1))/(childct-1))
#         print('int((stop-(start-1))/(childct-1))',step)
        
        step=min(step,limit)
        step=max(step,1)
        
#         print('itemdivs',itemdivs,'start',start,'stop',stop,'step',step,'limit',limit)
        
        nextkey=start
        
        for pk in range(start,stop,step):

            if pk>nextkey:
                vals=[v for v in range(nextkey,pk)]
                rangepicks.append(vals)
                nextkey=pk
            
        rangepicks.append([v for v in range(nextkey,stop+1)])

    return rangepicks

In [44]:
def gen_tree(prevkeylen,itemlen,keyset,currdivs,tempdict):
    
#     print('in gen_tree')
#     print('keyset',keyset)
#     print('currdivs',currdivs)
        
    prevkeylen=len(keyset)
    itemlen=len(currdivs)
    
    if prevkeylen>itemlen:
#         print('prevkeylen>itemlen')
        for i in range(itemlen):
            key=keyset[i]
            val=currdivs[i]
            tempdict[key]=val

    else:
#         print('else prevkeylen<=itemlen')
        step=itemlen//prevkeylen
        stepmod=itemlen % prevkeylen

#         print('step',step,'stepmod',stepmod)

        k=0
        i2=0

#         print('prevkeylen',prevkeylen,'itemlen',itemlen)
        
        for i in range(0,prevkeylen):
            key=keyset[i]
            start=i*step
            stop=start+step
            
            if i<prevkeylen-1:
#                 print('i<prevkeylen-1:')
                val=currdivs[start:stop]
            else:
#                 print('i>=prevkeylen-1:')
                val=currdivs[start:itemlen]
                
#             print('i,key,val',i,key,val)
            tempdict[key]=val
            
#     print('   ')
    

In [45]:
def write_sql_columns(treedict):
    
    templist=[]
    
    for k in treedict.keys():
            for v in treedict[k]:
                templist.append([k,v])
    return templist

In [46]:
def gen_divs(limit,branchcount,tempdict):
    
#     branchcount -- total number of nodes to assign in hierarchy
     # number of nodes currently assigned
    advance=0 # number of nodes that will be assigned in next pass
    residual=0
        
    rangmin=0
    keyct=1 #default
    rangemax=placecount=keyct # taking initial key as "most recent value used"
    childct=2
    
    
    itemdivs=[] # list of item divs
    
    keyset=[1]
    keysets=[] # list of key sets
    keysets.append(keyset)
    itemdivs.append(keyset)

    
    while placecount<=branchcount:
        
        if placecount>=branchcount:
            break
        
        keyct=len(keysets[len(keysets)-1]) # the current number of keys

        lastval=rangemax
        
        rangemin=lastval+1
        
        if (placecount+(keyct*childct))<=branchcount:
            advance=(keyct*childct)
            
        else:
#             print('final')
#             print('keysets',keysets)
            
            advance=(branchcount-lastval)
            rangemax=lastval+advance
            div=[rangemin,rangemax]
            currdivs=[i for i in range(rangemin,rangemax+1)]
            
            prevkeylen=len(keyset)
            itemlen=len(currdivs)
        
            gen_tree(prevkeylen,itemlen,keyset,currdivs,tempdict)
            
            itemdivs.append(currdivs)
            
            
            break
        
        rangemax=lastval+advance

        div=[rangemin,rangemax]

        currdivs=[i for i in range(rangemin,rangemax+1)]

        prevkeylen=len(keyset)
        itemlen=len(currdivs)
        
#         print('not final')
#         print('keysets',keysets)
        
        gen_tree(prevkeylen,itemlen,keyset,currdivs,tempdict)
        
        itemdivs.append(currdivs)
        
        keyct=min(keyct+1,limit)

        keyset=random.sample(range(rangemin,rangemax+1), keyct)

        keysets.append(keyset)
                 
        childct=min(childct+1,4)

        placecount=placecount+advance
    
    treeplan={}
    treeplan['keysets']=keysets
    treeplan['itemdivs']=itemdivs
    treeplan['dict']=tempdict
    
#     print('keysets',keysets)
#     print('itemdivs',itemdivs)
#     print('tempdict',tempdict)
    
    return treeplan
    
tempdict=AutoVivification()
treeplan=gen_divs(3,1000,tempdict)
text_for_sql=[]
text_for_sql=write_sql_columns(treeplan['dict'])

# with open('/Users/ouonomos/git_repo/reshift_dev/test_data/test_tree.json', 'w', encoding='utf-8') as f:
#     json.dump(treeplan['dict'], f, ensure_ascii=False, indent=4)
    
with open('text_for_sql.csv', 'w') as result:
    writer = csv.writer(result, delimiter="\t")
    writer.writerow(('mgrid', 'userid'))
    writer.writerows([i for i in row] for row in text_for_sql)


In [464]:
a=3 % 2
a

1

In [355]:
   
# store each group under prior group as key
    
#     childct=random.randint(0,3)

    # now for each child:            
#     for i in range(start,stop,2):
#             tempdict[placecount][i]=i+1
    
#     for k in tempdict[placecount].keys():
#         if (branchcount-placecount)>=2:
#             tempdict[placecount][k]=gen_tree(seed,branchcount,placecount,tempdict)

#     print('placecount',placecount)
    